In [2]:
# ======================================
# COMP0078 Coursework 2 Q1.7 RE1.1 (kNN)
# Douglas Chiang
# 15055142
# ======================================
cd /content/drive/MyDrive/Colab Notebooks/COMP0078/CW2/Q1

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/COMP0078/CW2/Q1'
/Users/DouglasChiang/Google Drive_HNC/_My Study/_UCL_Master_Robotics and Computation/Courses/_T1/COMP0078/Assignments/CW2/Code/Q1


In [3]:
ls

Assignment_2_Question_1_1_v3.ipynb
Assignment_2_Question_1_2_3_4_v1.ipynb
Assignment_2_Question_1_5_1_v1.ipynb
Assignment_2_Question_1_5_2_v1.ipynb
Assignment_2_Question_1_6_1_v1.ipynb
Assignment_2_Question_1_6_2_v1.ipynb
Assignment_2_Question_1_7_SVM_1_v1.ipynb
Assignment_2_Question_1_7_SVM_2_v1.ipynb
Assignment_2_Question_1_7_kNN_1_v1.ipynb
Assignment_2_Question_1_7_kNN_2_v1.ipynb
Assignment_2_Question_1bcd_v1.ipynb
plots/
zipcombo.dat


In [4]:
import time
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt
from   scipy             import stats

In [5]:
def load_dat(fname, Full_Data = False):
    dat = pd.read_csv(fname, sep='\s+', header=None).to_numpy()
    if Full_Data == False:
        y = dat[:, 0].astype(np.int) - 1  # we are zero-based indexing
    else:
        y = dat[:, 0].astype(np.int)
    
    X = dat[:, 1:].astype(np.float64)
    return X, y

In [6]:
def kNN(k, X, y, X_test):
    dot_product         = X_test.dot(X.T)
    X_train_norm        = np.sum(X**2,axis=-1)**(1./2)
    X_test_norm         = np.sum(X_test**2,axis=-1)**(1./2)
    norm_product        = np.multiply.outer(X_test_norm, X_train_norm)
    similarity          = dot_product/norm_product
    kN_candidate        = np.argsort(-similarity)[:,:k]
    kN_candidate_labels = np.squeeze(y[kN_candidate])
    if k == 1:
        pred = kN_candidate_labels
    else:
        pred = np.squeeze(stats.mode(kN_candidate_labels, axis = 1)[0].T)
    assert pred.shape[0] == X_test.shape[0]
    return pred

In [7]:
X_dataset, Y_dataset = load_dat("zipcombo.dat", Full_Data = True)
# split_pt = round(X_dataset.shape[0]*0.1)
# X_dataset = X_dataset[:split_pt]
# Y_dataset = Y_dataset[:split_pt]

In [8]:
# Question 1a):
Training_Error_Mean  = []
Training_Error_Stdev = []
Testing_Error_Mean   = []
Testing_Error_Stdev  = []

# ---------------Training--------------------------
# Question 1a):
# Loop for d from 1 to 7
K = [1,7,15,31,51,101,201]
for k in K:
    print("k = {}".format(k))

    # Question 1a:
    Training_Error_per_run = []
    Testing_Error_per_run  = []

    # 20 runs:
    for run in range(1,21):
        print("Run: {}".format(run))

        # Index Shuffle for fitting:
        ran_sample_idx       = np.random.permutation(len(Y_dataset))
        split_pt             = round(len(ran_sample_idx)*0.8)

        ran_train_sample_idx = ran_sample_idx[:split_pt]
        ran_test_sample_idx  = ran_sample_idx[split_pt:len(ran_sample_idx)]

        # Data sets:
        X_train              = X_dataset[ran_train_sample_idx]
        Y_train              = Y_dataset[ran_train_sample_idx]
        X_test               = X_dataset[ran_test_sample_idx]
        Y_test               = Y_dataset[ran_test_sample_idx]

        nsamples             = len(ran_train_sample_idx)
        n_test_samples       = len(ran_test_sample_idx)

        # Results memory:
        Train_correct_All    = []
        Train_accuracy_All   = []
        Train_mistakes_All   = []
        Test_correct_All     = []
        Test_accuracy_All    = []
        Test_mistakes_All    = []

        n_epochs = 1
        for epoch in range(1, n_epochs + 1):
            tic    = time.time()
            pred   = kNN(k, X_train, Y_train, X_train)
            toc    = time.time()

        # ---------------Training Accuracy-----------------
            Train_correct = 0
            Train_correct = sum(Y_train == pred)
            
            # Record results:
            Train_correct_All.append(Train_correct)

            Train_accuracy = Train_correct / nsamples
            Train_accuracy_All.append(Train_accuracy)

            Train_mistakes = nsamples - Train_correct
            Train_mistakes_All.append(Train_mistakes)

        # ---------------Testing Accuracy-------------------
            tic    = time.time()
            pred   = kNN(k, X_train, Y_train, X_test)
            toc    = time.time()
            Test_correct = 0
            Test_correct = sum(Y_test == pred)

            # Record results:
            Test_correct_All.append(Test_correct)

            Test_accuracy = Test_correct / n_test_samples
            Test_accuracy_All.append(Test_accuracy)

            Test_mistakes = n_test_samples - Test_correct
            Test_mistakes_All.append(Test_mistakes)

            print(f"**Training: Epoch {epoch} finished in {toc - tic:0.4f} seconds with Train accuracy {Train_accuracy:0.4f} and Test accuracy {Test_accuracy:0.4f}.")
            epoch += 1

        # Question 1a): Save error rate:
        Training_Error_per_run.append(Train_mistakes/nsamples)
        Testing_Error_per_run.append(Test_mistakes/n_test_samples)

    # Question 1a): Error mean and std:
    Training_Error_Mean.append(np.mean(Training_Error_per_run))
    Training_Error_Stdev.append(np.std(Training_Error_per_run))
    Testing_Error_Mean.append(np.mean(Testing_Error_per_run))
    Testing_Error_Stdev.append(np.std(Testing_Error_per_run))

print("Results:")
print(f"Training Error Mean: {Training_Error_Mean}")
print(f"Training Error STD:  {Training_Error_Stdev}")
print(f"Testing  Error Mean: {Testing_Error_Mean}")
print(f"Testing  Error STD:  {Testing_Error_Stdev}")

k = 1
Run: 1
**Training: Epoch 1 finished in 2.6225 seconds with Train accuracy 1.0000 and Test accuracy 0.9602.
Run: 2
**Training: Epoch 1 finished in 2.4964 seconds with Train accuracy 1.0000 and Test accuracy 0.9694.
Run: 3
**Training: Epoch 1 finished in 2.5034 seconds with Train accuracy 1.0000 and Test accuracy 0.9672.
Run: 4
**Training: Epoch 1 finished in 2.5452 seconds with Train accuracy 1.0000 and Test accuracy 0.9602.
Run: 5
**Training: Epoch 1 finished in 3.2705 seconds with Train accuracy 1.0000 and Test accuracy 0.9613.
Run: 6
**Training: Epoch 1 finished in 2.9436 seconds with Train accuracy 1.0000 and Test accuracy 0.9613.
Run: 7
**Training: Epoch 1 finished in 2.2063 seconds with Train accuracy 1.0000 and Test accuracy 0.9683.
Run: 8
**Training: Epoch 1 finished in 2.5634 seconds with Train accuracy 1.0000 and Test accuracy 0.9677.
Run: 9
**Training: Epoch 1 finished in 2.2522 seconds with Train accuracy 1.0000 and Test accuracy 0.9640.
Run: 10
**Training: Epoch 1 fin

**Training: Epoch 1 finished in 2.3979 seconds with Train accuracy 0.9353 and Test accuracy 0.9333.
Run: 18
**Training: Epoch 1 finished in 2.1172 seconds with Train accuracy 0.9348 and Test accuracy 0.9285.
Run: 19
**Training: Epoch 1 finished in 4.1741 seconds with Train accuracy 0.9359 and Test accuracy 0.9280.
Run: 20
**Training: Epoch 1 finished in 2.7590 seconds with Train accuracy 0.9359 and Test accuracy 0.9258.
k = 51
Run: 1
**Training: Epoch 1 finished in 2.3585 seconds with Train accuracy 0.9181 and Test accuracy 0.9075.
Run: 2
**Training: Epoch 1 finished in 2.7517 seconds with Train accuracy 0.9169 and Test accuracy 0.9156.
Run: 3
**Training: Epoch 1 finished in 2.6038 seconds with Train accuracy 0.9175 and Test accuracy 0.9108.
Run: 4
**Training: Epoch 1 finished in 2.5846 seconds with Train accuracy 0.9149 and Test accuracy 0.9237.
Run: 5
**Training: Epoch 1 finished in 3.8519 seconds with Train accuracy 0.9129 and Test accuracy 0.9242.
Run: 6
**Training: Epoch 1 finishe